# <p style="font-family: serif; font-size:2.2em; text-align: center;"><u>Nettoyage des données</u></p> 

---

# Chargement et description des données

In [11]:
pwd

'/home/simplon/Documents/Cliink_Equipe2/Jupyter'

In [21]:
# On importe la librairie pandas
import pandas as pd
# On modifie le nombre de lignes à afficher
pd.options.display.max_rows = 5
# Lecture dus fichiers
si = pd.read_csv("../Data/societes-immatriculees-2020.csv", index_col= 0, sep=';', skiprows=0)
data_villes = pd.read_csv("../Data/villes_data.csv", index_col= 0, sep=';', skiprows=0)
entreprises_par_sa = pd.read_csv("../Data/villes_data.csv", header=1 ,encoding="UTF-8", sep=',', skiprows=list(range(0, 2)) + list(range(14, 245)), usecols=[i for i in range(12)])
entreprises_par_sa_ef = pd.read_csv("../Data/villes_data.csv", header=16 ,encoding="UTF-8", sep=',', skiprows=list(range(15, 19)) + list(range(28, 245)), usecols=[i for i in range(31)])
année_2017 = pd.read_csv("../Data/villes_data.csv", header=30 ,encoding="UTF-8", sep=',', skiprows=list(range(29, 34)) + list(range(37, 245)), usecols=[i for i in range(14)])
pop_age = pd.read_csv("../Data/villes_data.csv", header=39 ,encoding="UTF-8", sep=',', skiprows=list(range(38, 41)) + list(range(50, 245)), usecols=[i for i in range(13)])
CSP_15_24 = pd.read_csv("../Data/villes_data.csv", header=84 ,encoding="UTF-8", sep=',', skiprows=list(range(83, 88)) + list(range(99, 245)), usecols=[i for i in range(6)])
CSP_25_54 = pd.read_csv("../Data/villes_data.csv", header=101 ,encoding="UTF-8", sep=',', skiprows=list(range(100, 105)) + list(range(116, 245)), usecols=[i for i in range(6)])
CSP_55_plus = pd.read_csv("../Data/villes_data.csv", header=118 ,encoding="UTF-8", sep=',', skiprows=list(range(117, 122)) + list(range(133, 245)), usecols=[i for i in range(6)])
emploi_femme = pd.read_csv("../Data/villes_data.csv", header=145 ,encoding="UTF-8", sep=',', skiprows=list(range(145, 150)) + list(range(161, 245)), usecols=[i for i in range(7)])
emploi_homme = pd.read_csv("../Data/villes_data.csv", header=162 ,encoding="UTF-8", sep=',', skiprows=list(range(162, 167)) + list(range(178, 245)), usecols=[i for i in range(7)])
chomage_sexe = pd.read_csv("../Data/villes_data.csv", header=179 ,encoding="UTF-8", sep=',', skiprows=list(range(179, 184)) + list(range(190, 245)), usecols=[i for i in range(7)])
taux_chomage_age = pd.read_csv("../Data/villes_data.csv", header=192 ,encoding="UTF-8", sep=',', skiprows=list(range(191, 196)) + list(range(202, 245)), usecols=[i for i in range(13)])
logement_par_catégorie = pd.read_csv("../Data/villes_data.csv", header=220 ,encoding="UTF-8", sep=',', skiprows=list(range(219, 224)) + list(range(230, 245)), usecols=[i for i in range(7)])
entreprise_csv = pd.read_csv("../Data/etablissements.csv")
# Lecture du fichier code naf
code_naf = pd.read_excel("../Data/naf2008_liste_n1.xls", skiprows=list(range(0, 2)))
# Lecture du fichier code naf niveau 5
naf_5 = pd.read_excel("../Data/naf2008_5_niveaux.xls")

/home/simplon/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (18,19,47,54,56,71) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


---

# Création des DataFrames

### Création du DataFrame sexe

In [22]:
liste_se_id = [1,2]
liste_se_nom = ["homme", "femme"]
sexe = pd.DataFrame({'se_id': liste_se_id,
                     'se_nom': liste_se_nom})

In [23]:
# Apercu du DataFrame sexe
sexe

,se_id,se_nom
0,1,homme
1,2,femme


### Création du DataFrame entreprise

In [24]:
# On affiche les adresses en minuscule
entreprise_csv["denominationUniteLegale"] = entreprise_csv["denominationUniteLegale"].str.lower()
# Rectification des code postaux
entreprise_csv["codePostalEtablissement"] = "0" + entreprise_csv["codePostalEtablissement"].astype(str)
# On affiche le nom des villes en minuscule
entreprise_csv["libelleCommuneEtablissement"] = entreprise_csv["libelleCommuneEtablissement"].str.lower()
# On affiche les abbreviations des types de voie en minuscule
entreprise_csv["typeVoieEtablissement"] = entreprise_csv["typeVoieEtablissement"].str.lower()
# On affiche les nom des rue en minuscule
entreprise_csv["libelleVoieEtablissement"] = entreprise_csv["libelleVoieEtablissement"].str.lower()

In [25]:
# Création du dictionnaire naf
naf = {}
for i in naf_5.index:
    # Ajout paire clé-valeur
    naf.update({naf_5.iloc[i]['NIV5']: naf_5.loc[i]['NIV1']})

# Création du dictionnaire secteur d'activité
SectAct = {}
for i in code_naf.index:
    # Ajout paire clé-valeur
    SectAct.update({code_naf.iloc[i]['Code']: code_naf.index[i]+1})

In [26]:
# Création du DataFrame entreprise
entreprise = pd.DataFrame(columns =['en_id','en_siren','en_nic', 'en_nom', 'en_sa_fk', 'en_adresse', 'en_code_postal', 'en_ville'])


#for i in entreprise_csv.index:
for i in range(2000):
    en_id = str(entreprise_csv.loc[i]["siren"]) + str(entreprise_csv.loc[i]["nic"])
    siren = entreprise_csv.iloc[i]['siren']
    nic = entreprise_csv.iloc[i]['nic']
    nom = entreprise_csv.iloc[i]['denominationUniteLegale']
    # Utilisation du dictionaire naf pour répertorier les secteurs d'activité
    a = naf[entreprise_csv.loc[i]["activitePrincipaleEtablissement"]]
    # Utilisation du dictionaire naf SectAct pour réferencer le secteur d'activité
    secteur_activite = SectAct[a]
    # Ici on test si le numero de voie est non nul/ si oui on le remplace par un str vide
    adresse = (''if pd.isna(entreprise_csv.iloc[i]['numeroVoieEtablissement'])
    # /sinon on le transforme en int pour supprimer les chiffres apres la virgule puis en str
 else str(int(entreprise_csv.iloc[i]['numeroVoieEtablissement'])))+" "+str(entreprise_csv.loc[i]["typeVoieEtablissement"])+" "+str(entreprise_csv.loc[i]["libelleVoieEtablissement"])
    code_postal = entreprise_csv.loc[i]["codePostalEtablissement"]
    ville = entreprise_csv.loc[i]["libelleCommuneEtablissement"]
    
    df = pd.DataFrame({'en_id': en_id,
                           'en_siren': siren,
                           'en_nic': nic,
                           'en_nom': nom,
                           'en_sa_fk': secteur_activite,
                           'en_adresse': adresse,
                           'en_code_postal':code_postal,
                           'en_ville':ville}, columns = ['en_id','en_siren', 'en_nic', 'en_nom', 'en_sa_fk', 'en_adresse', 'en_code_postal', 'en_ville'], index = [0])
    entreprise = entreprise.append(df, ignore_index=True)

In [28]:
# Apercu du DataFrame entreprise
entreprise

,en_id,en_siren,en_nic,en_nom,en_sa_fk,en_adresse,en_code_postal,en_ville
0,631096522,6310965,22,NaN,12,av gaston de fontmichel,06210,mandelieu-la-napoule
1,1558007934,15580079,34,societe immobiliere walter sperger,12,2 bd leader,06150,cannes
...,...,...,...,...,...,...,...,...
1998,32131425412,321314254,12,soc particuliere janau,12,70 rue georges clemenceau,06400,cannes
1999,32133932717,321339327,17,NaN,17,23 rue jean de riouffe,06400,cannes


### Création du DataFrame secteur_activite

In [19]:
# On utilise le fichier code naf pour le DataFrame secteur activité
secteur_activite = pd.DataFrame(code_naf, columns = ["sa_id", "Libellé"])
secteur_activite["sa_id"] = secteur_activite.index + 1
secteur_activite.rename(columns={"Libellé": "sa_nom"},inplace=True)

In [20]:
# Apercu du DataFrame esecteur_activite
secteur_activite

,sa_id,sa_nom
0,1,"Agriculture, sylviculture et pêche"
1,2,Industries extractives
...,...,...
19,20,Activités des ménages en tant qu'employeurs ; ...
20,21,Activités extra-territoriales


In [32]:
liste_ville_id = [1,2,3,4,5]
liste_ville_nom = ['Cannes', 'Le Cannet', 'Mougins', 'Mandelieu', 'Théoule sur Mer']
ville = pd.DataFrame({'vi_id': liste_ville_id,
                      'vi_nom' : liste_ville_nom })

In [33]:
ville

,vi_id,vi_nom
0,1,Cannes
1,2,Le Cannet
2,3,Mougins
3,4,Mandelieu
4,5,Théoule sur Mer


### Création du DataFrame sexe

In [29]:
liste_se_id = [1,2]
liste_se_nom = ["homme", "femme"]
sexe = pd.DataFrame({'se_id': liste_se_id,
                     'se_nom': liste_se_nom})

In [30]:
sexe

,se_id,se_nom
0,1,homme
1,2,femme
